In [4]:
from langchain.document_loaders import SitemapLoader

import bs4
from bs4 import BeautifulSoup
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

def docs_extractor(soup: BeautifulSoup) -> str:
    # Find and extract <h1> content
    h1_tag = soup.find('h1', class_='article-title')
    h1_content = h1_tag.text.strip() if h1_tag else "No <h1> found"
    # Find main content section
    section_tag = soup.find('section', class_='gh-content')
    final_content = ""
    for el in section_tag:
        if isinstance(el, bs4.element.Tag):
            tag = el.name
            if tag == "h1":
                final_content += f"# {el.text}\n"
            elif tag == "h2":
                final_content += f"## {el.text}\n"
            elif tag == "h3":
                final_content += f"### {el.text}\n"
            elif tag == "h4":
                final_content += f"#### {el.text}\n"
            elif tag == "h5":
                final_content += f"##### {el.text}\n"
            elif tag == "h6":
                final_content += f"###### {el.text}\n"
            elif tag == "p":
                final_content += f"{el.text}\n"
            elif tag == "ul":
                final_content += "- " + \
                    "\n- ".join(li.text for li in el.find_all('li')) + "\n"
            elif tag == 'b':
                final_content += f"**{el.text}**"
            elif tag == 'i':
                final_content += f"*{el.text}*"
            elif tag == 'pre' or tag == 'code':
                final_content += f"{el.text}\n"
            # Handle tables
            elif tag == 'table':
                table_content = ""
                for row in el.find_all('tr'):
                    row_content = ""
                    for cell in row.find_all(['td', 'th']):
                        row_content += cell.text + " | "
                    table_content += row_content + "\n"
                final_content += f"{table_content}\n"

            # Handle links
            elif tag == 'a':
                link_text = el.text
                link_url = el['href']
                final_content += f"[{link_text}]({link_url})"

    final_content = f"# {h1_content}\n\n{final_content}"
    return final_content.strip()


docs = SitemapLoader(
    "https://main--prisma-cloud-docs--hlxsites.hlx.live/docs/sitemaps/sitemap-en-0.xml",
    # filter_urls=['https://docs.prismacloud.io/en/enterprise-edition'],
    # parsing_function=docs_extractor,
).load()

Fetching pages: 0it [00:00, ?it/s]


In [5]:
docs

[]

In [ ]:
import xml.etree.ElementTree as ET

# Load the sitemap.xml file
sitemap_path = "sitemap-en-0.xml"  # Replace with the path to your sitemap.xml file
tree = ET.parse(sitemap_path)
root = tree.getroot()

# Define your base criteria
base_criteria = "https://docs.prismacloud.io/en/enterprise-edition"  # Replace with your base criteria

# Extract URLs that match the base criteria
matching_urls = []

for url in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
    if url.text.startswith(base_criteria):
        matching_urls.append(url.text)

# Print the matching URLs
for url in matching_urls:
    print(url)


In [7]:
len(matching_urls)

1739

In [9]:
with open("urls_to_be_scraped.txt",'w') as f:
    f.write("\n".join(matching_urls))

In [20]:
import requests
from bs4 import BeautifulSoup

def docs_extractor(soup: BeautifulSoup) -> str:
    # Find and extract all h1 headings on the page
    h1_headings = soup.find_all('h2')
    
    if h1_headings:
        for h1 in h1_headings:
            h1_content = h1.text.strip()
            print(f"H1 Heading: {h1_content}")
    else:
        print("No H1 headings found")
    # Find and extract <h1> content
    # h1_tag = soup.find('h1', class_='title')

    # h1_content = h1_tag.text.strip() if h1_tag else "No <h1> found"
    # return h1_content
    # # Find main content section
    # section_tag = soup.find('section', class_='gh-content')
    # final_content = ""
    # for el in section_tag:
    #     if isinstance(el, bs4.element.Tag):
    #         tag = el.name
    #         if tag == "h1":
    #             final_content += f"# {el.text}\n"
    #         elif tag == "h2":
    #             final_content += f"## {el.text}\n"
    #         elif tag == "h3":
    #             final_content += f"### {el.text}\n"
    #         elif tag == "h4":
    #             final_content += f"#### {el.text}\n"
    #         elif tag == "h5":
    #             final_content += f"##### {el.text}\n"
    #         elif tag == "h6":
    #             final_content += f"###### {el.text}\n"
    #         elif tag == "p":
    #             final_content += f"{el.text}\n"
    #         elif tag == "ul":
    #             final_content += "- " + \
    #                 "\n- ".join(li.text for li in el.find_all('li')) + "\n"
    #         elif tag == 'b':
    #             final_content += f"**{el.text}**"
    #         elif tag == 'i':
    #             final_content += f"*{el.text}*"
    #         elif tag == 'pre' or tag == 'code':
    #             final_content += f"{el.text}\n"
    #         # Handle tables
    #         elif tag == 'table':
    #             table_content = ""
    #             for row in el.find_all('tr'):
    #                 row_content = ""
    #                 for cell in row.find_all(['td', 'th']):
    #                     row_content += cell.text + " | "
    #                 table_content += row_content + "\n"
    #             final_content += f"{table_content}\n"

    #         # Handle links
    #         elif tag == 'a':
    #             link_text = el.text
    #             link_url = el['href']
    #             final_content += f"[{link_text}]({link_url})"

    # final_content = f"# {h1_content}\n\n{final_content}"
    # return final_content.strip()

# Replace the URL below with the link to the page you want to scrape
url = "https://docs.prismacloud.io/en/enterprise-edition/use-cases/secure-the-infrastructure/discover-sensitive-data-in-datastores"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    print(soup)
    # extracted_content = docs_extractor(soup)
    # Print or save the extracted content
    # print(extracted_content)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


<!DOCTYPE html>

<html>
<head>
<title>Metadata</title>
<link href="https://docs.prismacloud.io/en/enterprise-edition/use-cases/secure-the-infrastructure/discover-sensitive-data-in-datastores" rel="canonical"/>
<meta content="Metadata" property="og:title"/>
<meta content="https://docs.prismacloud.io/en/enterprise-edition/use-cases/secure-the-infrastructure/discover-sensitive-data-in-datastores" property="og:url"/>
<meta content="https://docs.prismacloud.io/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium" property="og:image"/>
<meta content="https://docs.prismacloud.io/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium" property="og:image:secure_url"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Metadata" name="twitter:title"/>
<meta content="https://docs.prismacloud.io/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium" name="twitter:image"/>
<meta content="enterprise-edition" name="product"/>
<meta

In [21]:
url="https://docs.prismacloud.io/en/enterprise-edition/use-cases/secure-the-infrastructure/tailor-prisma-cloud-to-match-your-security-needs"
try:
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP request errors
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Extract and print the data here
except requests.exceptions.RequestException as e:
    print("Request error:", e)
except Exception as e:
    print("Error:", e)


In [22]:
soup

<!DOCTYPE html>

<html>
<head>
<title>Metadata</title>
<link href="https://docs.prismacloud.io/en/enterprise-edition/use-cases/secure-the-infrastructure/tailor-prisma-cloud-to-match-your-security-needs" rel="canonical"/>
<meta content="Metadata" property="og:title"/>
<meta content="https://docs.prismacloud.io/en/enterprise-edition/use-cases/secure-the-infrastructure/tailor-prisma-cloud-to-match-your-security-needs" property="og:url"/>
<meta content="https://docs.prismacloud.io/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium" property="og:image"/>
<meta content="https://docs.prismacloud.io/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium" property="og:image:secure_url"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Metadata" name="twitter:title"/>
<meta content="https://docs.prismacloud.io/default-meta-image.png?width=1200&amp;format=pjpg&amp;optimize=medium" name="twitter:image"/>
<meta content="enterprise-edition" 

In [ ]:
from selenium import webdriver

# Create a new instance of the Chrome driver (you can use Firefox, etc.)
driver = webdriver.Chrome()

# Load the web page
driver.get("your_page_url_here")

# Wait for some time (optional) to ensure JavaScript content is loaded
# You may need to customize the wait time based on the page's loading behavior
driver.implicitly_wait(10)

# Get the page source, which includes the dynamic content
page_source = driver.page_source

# Now you can parse the page_source using Beautiful Soup as before
soup = BeautifulSoup(page_source, "html.parser")

# Extract specific elements from the HTML
title = soup.find("title").text

# Close the Selenium driver
driver.quit()

# Print the extracted information
print("Title:", title)


In [12]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [13]:
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader('hlxsites prisma-cloud-docs main docs-en_enterprise-edition', glob="**/*.adoc", loader_cls=TextLoader, use_multithreading=True, show_progress=True,loader_kwargs=text_loader_kwargs)

In [14]:
docs = loader.load()

100%|██████████| 1874/1874 [00:00<00:00, 2472.38it/s]


In [15]:
len(docs)

1834

In [16]:
docs[0]

Document(page_content='[#id0d4b823b-0b56-4562-9014-a03a0b12e67f]\n== Add a Resource List on Prisma Cloud\n\nUse tags to identify resources deployed in your cloud environments.\n\nA Resource List is a way to identify resources that are assigned with a specific tag or label. Resource lists can include tags or types of workloads.\n\n* xref:#id334bfb12-8cbe-460e-8698-fc4994e61b69[Identify Cloud Resources by Tags]\n* xref:#idd0ab9614-5daa-40b4-91cd-9bee6f70f2e6[Create a Resource List for Compute Resources]\n* xref:#id814aa2ea-b823-4727-93ea-010ccf9edd44[Create a Resource List for Azure Resource Groups]\n\n\n[.task]\n[#id334bfb12-8cbe-460e-8698-fc4994e61b69]\n=== Identify Cloud Resources by Tags\n\nA resource list for tags can reference tags that have been assigned to the resource as a part of a template deployment workflow or added manually. After you create the list to identify resources based on assigned tags, to use this list for scanning IaC templates using the Prisma Cloud plugins, you

In [6]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.pinecone import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from dotenv import load_dotenv
import os
load_dotenv()

pinecone.init(api_key=os.getenv("PINECONE_API_KEY"),environment=os.getenv("PINECONE_ENVIRONMENT"))

text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader('E:\\DESKTOP\\FreeLanceProjects\\powlosmillion\\hlxsites prisma-cloud-docs main docs-en_enterprise-edition', glob="**/*.adoc", loader_cls=TextLoader, use_multithreading=True, show_progress=True,loader_kwargs=text_loader_kwargs)

raw_docs = loader.load()



100%|██████████| 1874/1874 [00:00<00:00, 1939.12it/s]


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1300,
    chunk_overlap  = 50,
    length_function = len,
    is_separator_regex = False,
)

new_docs = text_splitter.split_documents(raw_docs)

In [8]:
len(new_docs)

5448

In [9]:
new_docs[0].page_content

'[#id0d4b823b-0b56-4562-9014-a03a0b12e67f]\n== Add a Resource List on Prisma Cloud\n\nUse tags to identify resources deployed in your cloud environments.\n\nA Resource List is a way to identify resources that are assigned with a specific tag or label. Resource lists can include tags or types of workloads.\n\n* xref:#id334bfb12-8cbe-460e-8698-fc4994e61b69[Identify Cloud Resources by Tags]\n* xref:#idd0ab9614-5daa-40b4-91cd-9bee6f70f2e6[Create a Resource List for Compute Resources]\n* xref:#id814aa2ea-b823-4727-93ea-010ccf9edd44[Create a Resource List for Azure Resource Groups]\n\n\n[.task]\n[#id334bfb12-8cbe-460e-8698-fc4994e61b69]\n=== Identify Cloud Resources by Tags\n\nA resource list for tags can reference tags that have been assigned to the resource as a part of a template deployment workflow or added manually. After you create the list to identify resources based on assigned tags, to use this list for scanning IaC templates using the Prisma Cloud plugins, you need to attach the re